# Example 1: Relaxation of H2O vibrational state under polynomial PES

| run type      | wavefunction | backend | Basis  | steps |
| ---           | ---          | ---     | ---    | ---   |
| improved relaxation | MPS-SM | Numpy   | HO-FBR | 20    |

<img src="../pic/h2o.png" alt="h2o" width="200">

## 1. Import modules

- Required in **any** calculations

In [1]:
from pytdscf import BasInfo, Model, Simulator

## 2. Set primitive basis as HO-FBR
- FBR = Analytical integral of orthonormal basis
- DVR = Numerical integral of Localized orthonormal basis

|     | implemented | Supported operator | Supported backend |
| --- | ---         | ---                | --- |
| FBR | HO          | polynomial of Q and P (sum of product; SOP) | Numpy (recommended), JAX|
| DVR | HO, Sine    | arbitrary function (matrix product operator; MPO) | Numpy, JAX (recommendend) |

In [2]:
from math import sqrt

from discvar import PrimBas_HO

from pytdscf import units
from pytdscf.potentials.h2o_potential import k_orig

freqs = [
    sqrt(k_orig[(1, 1)]) * units.au_in_cm1,
    sqrt(k_orig[(2, 2)]) * units.au_in_cm1,
    sqrt(k_orig[(3, 3)]) * units.au_in_cm1,
]  # a.u. (sqrt{omega^2} = omega)
freqs  # in cm^{-1}

[1658.474247654297, 3754.3152168607007, 3855.639069744621]

In [3]:
nprims = [9, 9, 9]  # Number of primitive basis
prim_info = [
    [
        PrimBas_HO(0.0, omega, nprim)
        for nprim, omega in zip(nprims, freqs, strict=False)
    ]
]
nstate = len(
    prim_info
)  # len(prim_info) is number of state, in this case state is S0
ndof = len(prim_info[0])  # Number of degree of freedom, H2O has 3 DOFs
basinfo = BasInfo(prim_info, spf_info=None)  # Set basis information object

**MPS-MCTDH wavefunction**
$$
|\Psi_{\rm{MPS-MCTDH}}\rangle = \sum_{\mathbf \{j\}}\sum_{\mathbf \{\tau\}}
a\substack{j_1 \\ 1\tau_1}a\substack{j_2 \\ \tau_1\tau_2} \cdots a\substack{j_f \\ \tau_{f-1}1}
|\varphi_{j_1}^{(1)}(q_1)\rangle|\varphi_{j_2}^{(2)}(q_2)\rangle
\cdots|\varphi_{j_f}^{(f)}(q_f)\rangle
$$
where SPF is 
$$
\varphi_{j_p}^{(p)}(q_p) = \sum_{i_p=1}^{n_p} c_{i_p}^{j_p}\chi_{i_p}^{(p)}(q_p) \; (j_p = 1,2,\ldots, N_p)
$$

Here, select $\{\chi_{i_p}^{(p)}(q_p)\}$ as Harmonic Oscillator eigenfunction.
See detail in [documenation](https://qclovers.github.io/PyTDSCF/pytdscf.html#pytdscf.primints_cls.PrimBas_HO).
Here one defines $n_p$ = 9, $N_p$ = 9. (i.e., Standard Method)

**NOTE**

- First argument of `Primbas_HO` is displaced dimensionless coordinate from q=0.0.

- In MPS,  $n = N$ (SM) is usually better than $n < M$ (MCTDH).  Only when using a laptop computer, MCTDH sometimes works better. (because the required RAM in MCTDH is smaller than SM.)

## 3. Set Hamiltonian (Polynomial Function)

- Here, one uses pre-calculated Polyonimal PES.
- `read_potential_nMR` includes kinetic terms as default.
- `k_orig` is the coefficients of Taylor expansion of PES in reference geometry by mass-weighted coordinate $Q_i$ (The unit of coordinate is **NOT AMU** but $\sqrt{m_e}a_0$, i.e. atomic units)
$$ V - V_0 = \frac{k_{11}}{2!} Q_1^2 + \frac{k_{22}}{2!} Q_2^2 + \frac{k_{33}}{2!} Q_3^2 + \frac{k_{111}}{3!} Q_1^3 + \frac{k_{122}}{2!} Q_1Q_2^2 + \cdots $$

In [4]:
k_orig

defaultdict(float,
            {(1, 1): 5.7101669772633975e-05,
             (2, 2): 0.00029261245707294615,
             (3, 3): 0.0003086200151857856,
             (1, 1, 1): -8.973542624563865e-07,
             (2, 2, 2): -1.8571147341445975e-05,
             (1, 2, 2): 5.028987089424822e-07,
             (1, 1, 2): 1.2870557913839666e-06,
             (1, 3, 3): 2.0063268625796784e-06,
             (2, 3, 3): -1.8853947560756764e-05,
             (1, 1, 1, 1): -2.2778131948543168e-08,
             (2, 2, 2, 2): 1.042951948572713e-06,
             (3, 3, 3, 3): 1.1133748664915738e-06,
             (1, 2, 2, 2): -8.193988329963448e-08,
             (1, 1, 2, 2): -1.852073688081903e-07,
             (1, 1, 1, 2): 5.750959195886642e-08,
             (1, 1, 3, 3): -2.1211138514059556e-07,
             (2, 2, 3, 3): 1.0721581542221527e-06,
             (1, 2, 3, 3): -1.256574051408931e-07})

In [5]:
from pytdscf.hamiltonian_cls import read_potential_nMR

hamiltonian = read_potential_nMR(k_orig, cut_off=-1.0e-10)
operators = {"hamiltonian": hamiltonian}

In [6]:
print("Onesite operators:", *hamiltonian.onesite[0][0])
print("Multisite operators:", *hamiltonian.general[0][0])

Onesite operators: -5.0000e-01 d^2_0 -5.0000e-01 d^2_1 -5.0000e-01 d^2_2 +2.8551e-05 q^2_0 +1.4631e-04 q^2_1 +1.5431e-04 q^2_2 -1.4956e-07 q^3_0 -3.0952e-06 q^3_1 -9.4909e-10 q^4_0 +4.3456e-08 q^4_1 +4.6391e-08 q^4_2
Multisite operators: +2.5145e-07 q^1_0 * q^2_1 +6.4353e-07 q^2_0 * q^1_1 +1.0032e-06 q^1_0 * q^2_2 -9.4270e-06 q^1_1 * q^2_2 -1.3657e-08 q^1_0 * q^3_1 -4.6302e-08 q^2_0 * q^2_1 +9.5849e-09 q^3_0 * q^1_1 -5.3028e-08 q^2_0 * q^2_2 +2.6804e-07 q^2_1 * q^2_2 -6.2829e-08 q^1_0 * q^1_1 * q^2_2


## 4. Set wavefunction (MPS) and All Model

- `m_aux_max` is a bond dimension of MPS (maximum rank of auxiliary index $\tau_p$)



In [7]:
model = Model(basinfo, operators)
model.m_aux_max = 9

## 5. Execute Calculation

- time step width is defined by `stepsize`=0.1 fs

F.Y.I., See also documentation about [Simulator](https://qclovers.github.io/PyTDSCF/pytdscf.html#pytdscf.simulator_cls.Simulator)

**NOTE**

- Runtype cannot be rebound. If you change the runtype, you should restart the kernel.

- Improved relaxation, i.e., diagonalization-based variational optimization, is much faster than pure imaginary time evolution.

- When simulating larger systems, (f>6, m>10), contraction of tensors can be overhead, in such a situation JAX, especially supporting GPU, is recommended to use as a backend.

In [8]:
jobname = "h2o_polynomial"
simulator = Simulator(jobname, model, ci_type="MPS", backend="Numpy", verbose=4)
simulator.relax(savefile_ext="_gs", maxstep=20, stepsize=0.1)

2025-01-16 17:03:37,465 - INFO:main.pytdscf._const_cls - 
     ____     __________   .____ ____   _____
    / _  |   /__  __/ _ \ / ___ / _  \ / ___/
   / /_) /_  __/ / / / ||/ /__ / / )_// /__
  /  ___/ / / / / / / / |.__  / |  __/ ___/
 /  /  / /_/ / / / /_/ /___/ /| \_/ / /
/__/   \__, /_/ /_____/_____/ \____/_/
      /____/

2025-01-16 17:03:37,465 - INFO:main.pytdscf._const_cls - Log file is ./h2o_polynomial_relax/main.log
2025-01-16 17:03:37,465 - INFO:main.pytdscf.simulator_cls - Set integral of FBR basis
2025-01-16 17:03:37,481 - INFO:main.pytdscf.simulator_cls - Set initial wave function (FBR basis)
2025-01-16 17:03:37,481 - INFO:main.pytdscf.simulator_cls - Prepare MPS w.f.
2025-01-16 17:03:37,482 - INFO:main.pytdscf._mps_cls - Initial MPS: 0-state with weights 1.0
2025-01-16 17:03:37,482 - INFO:main.pytdscf._mps_cls - Initial MPS: 0-state 0-mode with weight [     1.0000000000000      0.0000000000000      0.0000000000000      0.0000000000000      0.0000000000000      0.000000

(0.020855716615548497, <pytdscf.wavefunction.WFunc at 0x11d9023c0>)

## 6. Check Log file
See `h2o_polynomial_relax/main.log`, which is defined in `jobname`

In [9]:
!tail h2o_polynomial_relax/main.log

| pop 1.0000 | ene[eV]:  0.5675130 | time[fs]:    1.400 | elapsed[sec]:     0.19 | ci:  0.2  (ci_exp:  0.1|ci_rnm:  0.1|ci_etc:  0.0 d) |    0 MFLOPS (  0.0 s) 
| pop 1.0000 | ene[eV]:  0.5675130 | time[fs]:    1.500 | elapsed[sec]:     0.20 | ci:  0.2  (ci_exp:  0.1|ci_rnm:  0.1|ci_etc:  0.0 d) |    0 MFLOPS (  0.0 s) 
| pop 1.0000 | ene[eV]:  0.5675130 | time[fs]:    1.600 | elapsed[sec]:     0.21 | ci:  0.2  (ci_exp:  0.1|ci_rnm:  0.1|ci_etc:  0.0 d) |    0 MFLOPS (  0.0 s) 
| pop 1.0000 | ene[eV]:  0.5675130 | time[fs]:    1.700 | elapsed[sec]:     0.22 | ci:  0.2  (ci_exp:  0.1|ci_rnm:  0.1|ci_etc:  0.0 d) |    0 MFLOPS (  0.0 s) 
| pop 1.0000 | ene[eV]:  0.5675130 | time[fs]:    1.800 | elapsed[sec]:     0.23 | ci:  0.2  (ci_exp:  0.1|ci_rnm:  0.1|ci_etc:  0.0 d) |    0 MFLOPS (  0.0 s) 
Saved wavefunction    1.900 [fs]
| pop 1.0000 | ene[eV]:  0.5675130 | time[fs]:    1.900 | elapsed[sec]:     0.24 | ci:  0.2  (ci_exp:  0.1|ci_rnm:  0.1|ci_etc:  0.0 d) |    0 MFLOPS (  0.0 s) 
E

**Vibrational ground state energy is found to be `0.5675130` eV!**